In [1]:
import sys
sys.path.append('../')

In [2]:
from mrl.imports import *
from mrl.core import *
from mrl.chem import *
from mrl.templates import *

/home/dmai/miniconda3/envs/mrl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [3]:
from mrl.torch_imports import *
from mrl.torch_core import *
from mrl.layers import *
from mrl.dataloaders import *
from mrl.g_models import *
from mrl.agent import *
from mrl.vocab import *

In [47]:
df = pd.read_csv('../nbs/files/smiles.csv')
# vocab = CharacterVocab(SMILES_CHAR_VOCAB)

# vocab = CharacterVocab(AMINO_ACID_VOCAB)

vocab = FuncVocab(SELFIES_VOCAB, split_selfie,
                  prefunc=smile_to_selfie, postfunc=selfie_to_smile)

ds = Text_Dataset(df.smiles.values, vocab)
dl = ds.dataloader(16, num_workers=0)
loss = CrossEntropy()

# encoder = MLP_Encoder(2048, [1024, 512], 512, [0.1, 0.1])
# d_vocab = len(vocab.itos)
# d_embedding = 256
# d_hidden = 1024
# d_latent = 512
# n_layers = 3
# input_dropout = 0.3
# lstm_dropout = 0.3
# condition_hidden = True
# condition_output = False
# bos_idx = vocab.stoi['bos']
# norm_latent = True

# model = Conditional_LSTM_LM(encoder, 
#                                d_vocab, 
#                                d_embedding, 
#                                d_hidden, 
#                                d_latent, 
#                                n_layers,
#                                input_dropout, 
#                                lstm_dropout, 
#                                norm_latent,
#                                condition_hidden, 
#                                condition_output, 
#                                bos_idx)

# d_vocab = len(vocab.itos)
# d_embedding = 256
# d_hidden = 1024
# n_layers = 3
# input_dropout = 0.3
# lstm_dropout = 0.3
# bos_idx = vocab.stoi['bos']
# bidir = False
# tie_weights = True

# model = LSTM_LM(d_vocab, 
#                 d_embedding,
#                 d_hidden, 
#                 n_layers,
#                 input_dropout,
#                 lstm_dropout,
#                 bos_idx, 
#                 bidir, 
#                 tie_weights)

d_vocab = len(vocab.itos)
d_embedding = 256
encoder_d_in = 2048
encoder_dims = [1024, 512]
encoder_drops = [0.1, 0.1]
d_hidden = 1024
d_latent = 512
n_layers = 3
input_dropout = 0.3
lstm_dropout = 0.3
bidir = False
condition_hidden = True
condition_output = True
bos_idx = vocab.stoi['bos']

model = MLP_VAE(
            d_vocab,
            d_embedding,
            encoder_d_in,
            encoder_dims,
            encoder_drops,
            d_hidden,
            n_layers,
            d_latent,
            input_dropout=input_dropout,
            lstm_dropout=lstm_dropout,
            condition_hidden=condition_hidden,
            condition_output=condition_output,
            prior=None,
            bos_idx=bos_idx,
            transition=None,
        )

In [48]:
old_weights = torch.load('../nbs/untracked_files/fp_vae_chembl_selfies.pt')

In [49]:
new_weights = model.state_dict()

In [50]:
[v.shape for k,v in old_weights.items()]

[torch.Size([1024, 2048]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([]),
 torch.Size([512, 1024]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([512]),
 torch.Size([]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([79, 256]),
 torch.Size([4096, 768]),
 torch.Size([4096, 1024]),
 torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([4096, 1024]),
 torch.Size([4096, 1024]),
 torch.Size([4096]),
 torch.Size([4096]),
 torch.Size([1024, 1024]),
 torch.Size([1024, 256]),
 torch.Size([1024]),
 torch.Size([1024]),
 torch.Size([2048, 512]),
 torch.Size([2048]),
 torch.Size([2048, 512]),
 torch.Size([2048]),
 torch.Size([512, 512]),
 torch.Size([512]),
 torch.Size([79, 256]),
 torch.Size([79]),
 torch.Size([1024, 512]),
 torch.Size([1024]),
 torch.Size([512]),
 torch.Size([512])]

In [51]:
old_keys = list(old_weights.keys())
new_keys = list(new_weights.keys())

for i in range(len(old_keys)):
    new_weights[new_keys[i]] = old_weights[old_keys[i]]

In [52]:
model.load_state_dict(new_weights)

<All keys matched successfully>

In [53]:
to_device(model);

In [54]:
model.eval();

In [55]:
preds, _ = model.sample_no_grad(512, 90)

In [56]:
smiles = [vocab.reconstruct(i) for i in preds]

In [57]:
mols = [to_mol(i) for i in smiles]
# mols = [to_protein(i) for i in smiles]

In [58]:
len([i for i in mols if i is not None])/len(mols)

0.98828125

In [59]:
len(set(smiles))/len(smiles)

0.990234375

In [60]:
torch.save(model.state_dict(), '../nbs/untracked_files/fp_vae_chembl_selfies.pt')

In [ ]:
class PoolingHead(nn.Module):
    def __init__(self, d_in, dims, d_out, drops, outrange=None):
        super().__init__()
        
        self.layers = MLP_Encoder(d_in, dims, d_out, drops)
        self.outrange = outrange
        
    def forward(self, x, mask=None):
        # x - bs, sl, d
        # mask - bs, sl
        if mask is not None:
            lengths = mask.sum(-1)
            final_vals = x[torch.arange(x.shape[0]), lengths-1]
            pool1 = x.masked_fill(mask.unsqueeze(-1), 0).sum(1)/lengths.unsqueeze(-1)
            pool2 = x.masked_fill(mask.unsqueeze(-1), -float('inf')).max(1)[0]
        else:
            final_vals = x[:,-1]
            pool1 = x.mean(1)
            pool2 = x.max(1)[0]
            
        x = torch.cat([final_vals, pool1, pool2], 1)
        x = self.layers(x)
        
        if self.outrange is not None:
            x = torch.sigmoid(x) * (self.outrange[1]-self.outrange[0]) + self.outrange[0]
        
        return x
    
class Predictive_LSTM(nn.Module):
    def __init__(self, d_vocab, d_embedding, d_hidden, n_layers, d_out_lstm,
                 head_dims, head_drops, d_out, outrange=None,
                 lstm_drop=0., bidir=False,):
        super().__init__()
        
        self.embedding = nn.Embedding(d_vocab, d_embedding)
        self.lstm = LSTM(d_embedding, d_hidden, d_out_lstm, n_layers,
                                     bidir=bidir, dropout=lstm_drop)
        
        self.head = PoolingHead(d_out_lstm*3, head_dims, d_out, head_drops, outrange)

        
    def forward(self, x, hiddens=None, mask=None):
        
        x = self.embedding(x)
        encoded, hiddens = self.lstm(x, hiddens)
        output = self.head(encoded, mask)
        return output
    
    def freeze_encoder(self):
        for p in self.embedding.parameters():
            p.requires_grad_(False)
            
        for p in self.lstm.parameters():
            p.requires_grad_(False)
    
    def load_from_lm(self, lm_model):
        if hasattr(lm_model, 'block'):
            self.embedding.load_state_dict(lm_model.block.embedding.state_dict())
            self.lstm.load_state_dict(lm_model.block.lstm.state_dict())
        else:
            if hasattr(lm_model, 'lstm'):
                self.lstm.load_state_dict(lm_model.lstm.state_dict())
                
            if hasattr(lm_model, 'embedding'):
                self.embedding.load_state_dict(lm_model.embedding.state_dict())

In [ ]:
vocab = CharacterVocab(SMILES_CHAR_VOCAB)

In [ ]:
ds = TextDataset(['CCC'], vocab)

d_vocab = len(vocab.itos)
d_embedding = 256
d_hidden = 1024
n_layers = 3
lstm_drop = 0.
lin_drop = 0.
bos_idx = vocab.stoi['bos']
bidir = False
tie_weights = True

lm_model = LSTM_LM(d_vocab, d_embedding, d_hidden, n_layers,
                lstm_drop, lin_drop, bos_idx, bidir, tie_weights)

lm_model.load_state_dict(torch.load('../nbs/untracked_files/lstm_lm_small.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model = Predictive_LSTM(d_vocab, d_embedding, d_hidden, n_layers, d_embedding,
                        [512, 256], [0.1, 0.1], 1)

In [ ]:
model.load_from_lm(lm_model)

In [ ]:
next(model.lstm.parameters())

Parameter containing:
tensor([[ 0.0181,  0.0324,  0.0202,  ...,  0.0010,  0.0047,  0.0283],
        [-0.0101,  0.0228,  0.0193,  ..., -0.0366,  0.0038, -0.0540],
        [ 0.0537, -0.0437, -0.0073,  ...,  0.0270, -0.0121,  0.0504],
        ...,
        [ 0.0567,  0.0338, -0.0113,  ..., -0.0163,  0.0104,  0.0140],
        [ 0.0025,  0.0101,  0.0410,  ..., -0.0130, -0.0164, -0.0144],
        [ 0.0174,  0.0205,  0.0073,  ...,  0.0442, -0.0021,  0.0154]],
       requires_grad=True)

current roadmap
    start big rewrite
        do reward functions
        revisit mol/protein distinctions in template/block
    buffer changes
        decide rewards will be nondifferentiable, string derived, and hashed
        build buffer
        score buffer
        on batch
            live sample
            score live sample
            buffer sample (scored)
            get model outputs
            continue
    now we do documentation/update site
    combichem
    jtnnvae
    start working on other docs
        guides
        examples
    move to release v0
    


environment    
callbacks (base)
log/buffer callbacks
samplers
agent
rewards
losses
policy gradient

callbacks
    core
    logging
    template
    
agent
PG
losses
samplers
rewards





    

future modules
    combichem
    pharmacophore
    active learning
    new models
        jtnnvae
        flow based
    chemprop hooks
    huggingface hooks
        
project quality
    collab links
    google search on site
    more documentation

Use cases/examples
    basic drug design
    iterative drug design with dataset building
    drug design with docking proxy
    proteins/antibodies
    polymers
    catalyst
    materials
    
guides
    generative screening primer
    basics
        stuff about how bad score functions can be
    how to use callbacks
    basic drug design
    contrastive optimization
    latent/prior optimization
    training tips/tricks (callback stuff)
    
paapers to implement
    fastai guy - conditional lstm, contrastive
    moldqn - dqn sampler (rollout), agent is score function, dqn loss
    synthesis constrained? really same as DQN with different rolllout
    stoned selfies/genetic algorithm examples
    deep drug decoder/paul paper/other conditional generation